# SENTIMENT ANALYSIS

In [1]:
import random
class Sentiment:
    NEGATIVE='NEGATIVE'
    NEUTRAL='NEUTRAL'
    POSITIVE='POSITIVE'
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentimethent=self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
class ReviewContainer:
    #even out
    def __init__(self,reviews):
        self.reviews=reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive = list(filter(lambda x: x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)
        
        

In [2]:
#load data
import json
file_name='Books_small_10000.json'
reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))
reviews[5].text
        

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [3]:
#PrepData
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.33,random_state=42)
train_cont= ReviewContainer(training)
test_cont= ReviewContainer(test)


In [4]:
#bags of words 
train_cont.evenly_distribute()
train_x= train_cont.get_text()
train_y= train_cont.get_sentiment()
test_cont.evenly_distribute()
test_x=test_cont.get_text()
test_y=test_cont.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

AttributeError: 'Review' object has no attribute 'sentiment'

In [ ]:
#Bag of words vectorization
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer=TfidfVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)
print(train_x[0])
print(train_x_vectors[0].toarray())


#Classification--to fit to model



In [ ]:
#linear SVM
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)
test_x[0]
clf_svm.predict(test_x_vectors[0])

In [ ]:
#decison tree
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
clf_dec.predict(test_x_vectors[0])

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors,train_y)
clf_log.predict(test_x_vectors[0])

#Evaluation


In [ ]:
#mean accuracy
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))

In [ ]:
#f1 score: model is biased because there are too many positives..we e=need a balanced data 
from sklearn.metrics import f1_score
f1_svm=f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])
f1_dec=f1_score(test_y,clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])
f1_log=f1_score(test_y,clf_log.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])

print(f1_svm)
print(f1_dec)
print(f1_log)



In [ ]:
#Qualitative Analysis 
test_set=['not great','bad book do not buy','horrible waste if time']
new_test= vectorizer.transform(test_set)
clf_svm.predict(new_test)

###Tuning our model (with Grid Search)

In [ ]:
from sklearn.model_selection import GridSearchCV
params={'kernel':('linear','rbf'),'C':(1,4,8,16,32)}
svc=svm.SVC()
clf=GridSearchCV(svc,params,cv=5)
clf.fit(train_x_vectors,train_y)


In [ ]:
print(clf.score(test_x_vectors,test_y))

In [ ]:
#Saving Model
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
#Load Model
import pickle
with open('model.pkl','rb') as f:
    loaded_clf=pickle.load(f)
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])